In [1]:
import pandas as pd
from functools import reduce
import numpy as np

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier

In [2]:
feature_dir = 'data/features/'

In [3]:
df_twitter = pd.read_csv(feature_dir+'twitter_features.csv')
df_news = pd.read_csv(feature_dir+'news_features.csv')
df_out= pd.read_csv(feature_dir+'index_features.csv')

In [4]:
clf = SVC(C=1.0)
clf2 = SVR(C=1.0)

In [5]:
df_y = df_out
df_X = pd.merge(df_twitter, df_news, on='Date', how='outer')

In [6]:
df_y

,Date,Change VIX,Direction VIX,Lag 1 Change VIX,Lag 1 Direction VIX,Lag 2 Change VIX,Lag 2 Direction VIX,Lag 2 Significant VIX,Lag 5 Change VIX,Lag 5 Direction VIX,Change SPY,Direction SPY,Lag 1 Change SPY,Lag 1 Direction SPY,Lag 2 Change SPY,Lag 2 Direction SPY,Lag 2 Significant SPY,Lag 5 Change SPY,Lag 5 Direction SPY
0,2015-01-02,0.116279,1,0.080518,1,0.144707,1,1,0.144707,1,-0.007741,-1,-0.010708,-1,-0.020787,-1,-1,-0.020787,-1
1,2015-01-05,0.080518,1,0.059406,1,0.050026,1,1,0.050026,1,-0.010708,-1,-0.010188,-1,-0.013469,-1,-1,-0.013469,-1
2,2015-01-06,0.059406,1,-0.008854,-1,-0.118052,-1,-1,-0.118052,-1,-0.010188,-1,-0.003315,-1,0.009501,1,0,0.009501,1
3,2015-01-07,-0.008854,-1,-0.110174,-1,-0.184119,-1,-1,-0.184119,-1,-0.003315,-1,0.012859,1,0.024724,1,1,0.024724,1
4,2015-01-08,-0.110174,-1,-0.083101,-1,0.005020,1,0,0.005020,1,0.012859,1,0.011715,1,0.001961,1,0,0.001961,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2019-04-24,-0.010269,-1,0.060654,1,0.072626,1,1,0.072626,1,0.007259,1,-0.002288,-1,-0.002357,-1,0,-0.002357,-1
1084,2019-04-25,0.060654,1,0.011287,1,-0.018811,-1,-1,-0.018811,-1,-0.002288,-1,-0.000068,-1,0.004758,1,0,0.004758,1
1085,2019-04-26,0.011287,1,-0.029762,-1,-0.024554,-1,-1,-0.024554,-1,-0.000068,-1,0.004827,1,0.004759,1,0,0.004759,1
1086,2019-04-29,-0.029762,-1,0.005368,1,-0.013804,-1,-1,-0.013804,-1,0.004827,1,-0.000068,-1,0.004123,1,0,0.004123,1


In [7]:
df_X

,Date,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score,Trump_sentiment_score,Trump_heuristic_score,New York Times_sentiment_score,New York Times_polar_score,CNN_sentiment_score,CNN_polar_score,Washington Post_sentiment_score,Washington Post_polar_score,NYT_sentiment_score,NYT_polar_score
0,2015-01-02,-0.107225,-0.114245,NaN,NaN,NaN,NaN,0.323554,0.331358,NaN,NaN,-0.176336,-0.1492,NaN,NaN,-0.012855,-0.0970
1,2015-01-05,0.610250,0.628957,NaN,NaN,NaN,NaN,0.352101,0.383644,NaN,NaN,-0.281860,-0.8519,NaN,NaN,0.021248,0.0937
2,2015-01-06,0.160900,0.549501,NaN,NaN,0.193586,0.403680,0.354565,0.295516,NaN,NaN,0.100810,0.1651,NaN,NaN,-0.018193,-0.1756
3,2015-01-07,0.469467,0.445845,NaN,NaN,0.183080,0.247557,0.109130,-0.421761,NaN,NaN,-0.229275,-0.4767,NaN,NaN,-0.045886,0.1026
4,2015-01-08,0.370700,0.621998,NaN,NaN,0.571667,0.673277,0.099112,-0.122951,NaN,NaN,-0.360086,-0.7003,NaN,NaN,-0.034352,-0.0730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2019-04-24,NaN,NaN,NaN,NaN,NaN,NaN,0.252729,0.244343,NaN,NaN,NaN,NaN,NaN,NaN,-0.050849,-0.1125
1084,2019-04-25,NaN,NaN,NaN,NaN,NaN,NaN,-0.215995,0.026744,NaN,NaN,NaN,NaN,NaN,NaN,-0.028734,0.0418
1085,2019-04-26,NaN,NaN,NaN,NaN,NaN,NaN,0.192100,0.323469,NaN,NaN,NaN,NaN,NaN,NaN,-0.078153,-0.2229
1086,2019-04-29,NaN,NaN,NaN,NaN,NaN,NaN,0.354891,0.354788,NaN,NaN,NaN,NaN,NaN,NaN,-0.126724,-0.1281


In [8]:
df_X.columns

Index(['Date', 'BarackObama_sentiment_score', 'BarackObama_heuristic_score',
       'cnnbrk_sentiment_score', 'cnnbrk_heuristic_score',
       'KimKardashian_sentiment_score', 'KimKardashian_heuristic_score',
       'Trump_sentiment_score', 'Trump_heuristic_score',
       'New York Times_sentiment_score', 'New York Times_polar_score',
       'CNN_sentiment_score', 'CNN_polar_score',
       'Washington Post_sentiment_score', 'Washington Post_polar_score',
       'NYT_sentiment_score', 'NYT_polar_score'],
      dtype='object')

In [9]:
X = df_X.reset_index(drop=True)[['NYT_sentiment_score', 'Trump_sentiment_score']]
y = df_y.reset_index(drop=True)
# X = X.drop(columns=['Date'])
X.fillna(0, inplace=True)
y = y['Lag 2 Significant VIX']
# y.fillna(y.mean(), inplace=True)

In [10]:
np.isinf(X).all()

NYT_sentiment_score      False
Trump_sentiment_score    False
dtype: bool

In [11]:
X

,NYT_sentiment_score,Trump_sentiment_score
0,-0.012855,0.323554
1,0.021248,0.352101
2,-0.018193,0.354565
3,-0.045886,0.109130
4,-0.034352,0.099112
...,...,...
1083,-0.050849,0.252729
1084,-0.028734,-0.215995
1085,-0.078153,0.192100
1086,-0.126724,0.354891


In [12]:
y

0       1
1       1
2      -1
3      -1
4       0
       ..
1083    1
1084   -1
1085   -1
1086   -1
1087    1
Name: Lag 2 Significant VIX, Length: 1088, dtype: int64

In [13]:
y.value_counts()

-1    526
 1    455
 0    107
Name: Lag 2 Significant VIX, dtype: int64

In [14]:
# Guessing Most common class always
succ = y.value_counts().max()/(y.count())

In [15]:
succ

0.4834558823529412

In [16]:
np.where(np.isnan(X))

(array([], dtype=int32), array([], dtype=int32))

In [17]:
y

0       1
1       1
2      -1
3      -1
4       0
       ..
1083    1
1084   -1
1085   -1
1086   -1
1087    1
Name: Lag 2 Significant VIX, Length: 1088, dtype: int64

In [18]:
def run_experiment(model, X, y):
    scores = []
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = X.loc[train_index],X.loc[test_index], y.loc[train_index], y.loc[test_index]
        X_train.fillna(X_train.mean(), inplace=True)
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
    print('Scores',scores)
    print('Average score', sum(scores)/len(scores))

run_experiment(clf, X, y)

Scores [0.4954128440366973, 0.463302752293578, 0.44495412844036697, 0.47465437788018433, 0.5299539170506913]
Average score 0.4816556039403036


In [87]:
X = df_X.reset_index(drop=True).set_index('Date')[['NYT_sentiment_score']]#Trump_sentiment_score
y = df_y.reset_index(drop=True).set_index('Date')
X.fillna(0, inplace=True)
y = y['Lag 2 Significant VIX']

df_X.reset_index(drop=True).set_index('Date')

,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score,Trump_sentiment_score,Trump_heuristic_score,New York Times_sentiment_score,New York Times_polar_score,CNN_sentiment_score,CNN_polar_score,Washington Post_sentiment_score,Washington Post_polar_score,NYT_sentiment_score,NYT_polar_score
Date,,,,,,,,,,,,,,,,
2015-01-02,-0.4289,-3.145348,NaN,NaN,NaN,NaN,31.0612,106.794357,NaN,NaN,-6.3481,-0.1492,NaN,NaN,-8.5101,-0.0970
2015-01-05,1.2205,9.206268,NaN,NaN,NaN,NaN,49.6462,177.616544,NaN,NaN,-1.4093,-0.8519,NaN,NaN,9.4768,0.0937
2015-01-06,0.6436,7.880455,NaN,NaN,4.0653,37.652221,37.9385,114.107390,NaN,NaN,1.0081,0.1651,NaN,NaN,-5.3850,-0.1756
2015-01-07,5.6336,40.112804,NaN,NaN,0.9154,8.877281,3.2739,2.292712,NaN,NaN,-0.9171,-0.4767,NaN,NaN,-23.5856,0.1026
2015-01-08,1.1121,9.476567,NaN,NaN,1.7150,16.294419,4.9556,12.653774,NaN,NaN,-2.5206,-0.7003,NaN,NaN,-9.8248,-0.0730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-24,NaN,NaN,NaN,NaN,NaN,NaN,4.2964,47.781231,NaN,NaN,NaN,NaN,NaN,NaN,-9.0002,-0.1125
2019-04-25,NaN,NaN,NaN,NaN,NaN,NaN,-4.5359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.8098,0.0418
2019-04-26,NaN,NaN,NaN,NaN,NaN,NaN,5.7630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-30.7923,-0.2229


In [94]:
import datetime
import calendar
from dateutil.relativedelta import relativedelta

def run_experiment2(model, X, y):
    scores = []
    
    train_start = datetime.date(2015, 1, 1)
    train_end = (train_start + relativedelta(months=2))
    days = calendar.monthrange(train_end.year, train_end.month)[1]
    train_end = train_end.replace(day=days)
    test_start = (train_start + relativedelta(months=3)).replace(day=1)
    test_end = (train_start + relativedelta(months=3)).replace(day=14)
    
    while train_end < datetime.date(2019, 4, 1):
        X_train = X.loc[str(train_start):str(train_end)]
        X_train = X_train.fillna(X_train.mean())
        y_train = y.loc[str(train_start):str(train_end)]
        
        X_test = X.loc[str(test_start):str(test_end)]
        y_test = y.loc[str(test_start):str(test_end)]
        
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
        train_start = (train_start + relativedelta(months=1))
        train_end = (train_start + relativedelta(months=2))
        days = calendar.monthrange(train_end.year, train_end.month)[1]
        train_end = train_end.replace(day=days)
        test_start = (train_start + relativedelta(months=3)).replace(day=1)
        test_end = (train_start + relativedelta(months=3)).replace(day=14)
    print('Scores',scores)
    print('Average score', sum(scores)/len(scores))

run_experiment2(clf, X, y)

Scores [0.7777777777777778, 0.1, 0.6, 0.5555555555555556, 0.4, 0.4444444444444444, 0.5, 0.5, 0.5, 0.3333333333333333, 0.4, 0.2, 0.5, 0.5, 0.1, 0.0, 0.4, 0.1111111111111111, 0.4, 0.6, 0.5, 0.3333333333333333, 0.2, 0.2, 0.4444444444444444, 0.4, 0.4, 0.6666666666666666, 0.5, 0.5555555555555556, 0.4, 0.4, 0.6, 0.6666666666666666, 0.5, 0.7, 0.5, 0.7, 0.4, 0.7777777777777778, 0.6, 0.5555555555555556, 0.3, 0.7, 0.6666666666666666, 0.5555555555555556, 0.4, 0.5, 0.5]
Average score 0.4600907029478458


In [260]:
def rfc_param_selection(X, y):
    n_estims= [200, 700]
    max_feats= ['auto', 'sqrt', 'log2']
    param_grid = {'n_estimators': n_estims, 'max_features' : max_feats}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [256]:
def svc_param_selection(X, y):
    kernels = ['rbf','linear']
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel':kernels}
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [257]:
def MLP_param_selection(X, y):
    solvers = ['lbfgs', 'sgd']
    max_iters = [1000,1500,2000 ]
    alphas = [10,100,1000,10000]
    hidden_layer_sizes = [10,12,15]
    random_states = [i for i in range(10)]
    param_grid = {'solver': solvers, 'max_iter': max_iters, 'alpha': alphas, 'hidden_layer_sizes':hidden_layer_sizes, 'random_state':random_states}
    

    grid_search = GridSearchCV(MLPClassifier(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_


In [258]:
svc_param_selection(X,y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.486, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV] . C=0.001, gamma=0.001, kernel=linear, score=0.486, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV] . C=0.001,

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.486, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=linear ..............................
[CV] .. C=0.001, gamma=0.01, kernel=linear, score=0.486, total=   0.0s
[CV] C=0.001, gamma=0.01, kernel=linear ..............................
[CV] .. C=0.001, gamma=0.01, kernel=linear, score=0.482, total=   0.0s
[CV] C

[CV] .... C=0.01, gamma=0.1, kernel=linear, score=0.484, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] .... C=0.01, gamma=0.1, kernel=linear, score=0.484, total=   0.0s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.486, total=   0.0s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.482, total=   0.0s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.484, total=   0.0s
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] .

[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.482, total=   0.1s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.482, total=   0.1s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.484, total=   0.1s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.484, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.482, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.505, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.454, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] .

[CV] ...... C=10, gamma=0.1, kernel=linear, score=0.486, total=  45.2s
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] ...... C=10, gamma=0.1, kernel=linear, score=0.482, total=  46.9s
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] ...... C=10, gamma=0.1, kernel=linear, score=0.482, total=  36.5s
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] ...... C=10, gamma=0.1, kernel=linear, score=0.484, total=  32.0s
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] ...... C=10, gamma=0.1, kernel=linear, score=0.484, total=  24.2s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.422, total=   0.0s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.399, total=   0.0s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 12.7min finished


{'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}

In [259]:
rfc_param_selection(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   24.9s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.4311081046801674


{'max_features': 'log2', 'n_estimators': 200}

In [19]:
model = RandomForestClassifier(max_features='log2',n_estimators=200)
run_experiment(model, X, y)

Scores [0.4954128440366973, 0.42201834862385323, 0.44954128440366975, 0.45161290322580644, 0.42857142857142855]
Average score 0.44943136177229104


In [20]:
model = SVC(C=1, gamma=.1, kernel='rbf')
run_experiment(model, X, y)

Scores [0.4954128440366973, 0.46788990825688076, 0.44495412844036697, 0.4930875576036866, 0.5161290322580645]
Average score 0.4834946941191392


In [262]:
MLP_param_selection(X,y)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits
[CV] alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs, score=0.491, total=   0.4s
[CV] alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs 
[CV]  alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs, score=0.463, total=   0.2s
[CV] alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


[CV]  alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs, score=0.450, total=   0.3s
[CV] alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs 
[CV]  alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs, score=0.470, total=   0.2s
[CV] alpha=10, hidden_layer_sizes=10, max_iter=1000, random_state=0, solver=lbfgs 


...

[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd, score=0.486, total=   0.2s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd, score=0.482, total=   0.2s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd, score=0.482, total=   0.2s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd, score=0.484, total=   0.2s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=2, solver=sgd, score=0.484, total=   0.2s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=3, solver=lbfgs 
[CV]  alpha=10000,

[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=6, solver=sgd, score=0.482, total=   0.3s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=6, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=6, solver=sgd, score=0.484, total=   0.3s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=6, solver=sgd 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=6, solver=sgd, score=0.484, total=   0.3s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=7, solver=lbfgs 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=7, solver=lbfgs, score=0.486, total=   0.0s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=7, solver=lbfgs 
[CV]  alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=7, solver=lbfgs, score=0.482, total=   0.0s
[CV] alpha=10000, hidden_layer_sizes=15, max_iter=2000, random_state=7, solver=lbfgs 
[CV]  alph

[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed: 15.5min finished


MLPClassifier(activation='relu', alpha=10, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=10, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
0.4880649389083837


{'alpha': 10,
 'hidden_layer_sizes': 10,
 'max_iter': 1000,
 'random_state': 1,
 'solver': 'sgd'}

In [21]:
model = MLPClassifier(solver='lbfgs', alpha=10, max_iter=1500, hidden_layer_sizes=10)
run_experiment(model, X, y)

Scores [0.4954128440366973, 0.46788990825688076, 0.44495412844036697, 0.4930875576036866, 0.5161290322580645]
Average score 0.4834946941191392


In [ ]:
print(3)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
auto_y = df_out['Change SPY']

In [ ]:
auto_y

In [ ]:
plot_acf(auto_y, lags = 50)